**🔹 Installed Required dependencies**

In [ ]:
# # **Clustering Text Using BERT Embeddings, Saving Embeddings, and Visualizing Clusters**

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
!pip uninstall torch torch_xla -y
!pip install torch transformers


Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl (797.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cpu requires torch==2.3.0, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.0+cpu requires torch==2.3.0, but you have torch 2.4.0 which is incompatible.


**🔹 Necessary libray Used**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
import torch
import r
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import BertTokenizer, BertModel

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Define file paths
train_file_path = '/content/train.parquet'
test_file_path = '/content/test.parquet'

# Load the datasets
train_df = pd.read_parquet(train_file_path)
test_df = pd.read_parquet(test_file_path)

# Display the first few rows to understand the structure
print(train_df.head())
print(test_df.head())

# Initialize NLTK tools
nltk.download('stopwords')
nltk.download('wordnet')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define text preprocessing functions
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    text = text.lower()  # Convert to lowercase
    words = text.split()  # Split into words
    words = [word for word in words if word not in stop_words]  # Remove stop words
    return ' '.join(words)

def normalize_text(text, method='lemmatization'):
    words = text.split()
    if method == 'stemming':
        words = [stemmer.stem(word) for word in words]
    elif method == 'lemmatization':
        words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

def preprocess_text(text, method='lemmatization'):
    text = clean_text(text)
    text = normalize_text(text, method)
    return text

# Apply preprocessing to the train and test data
if 'text' in train_df.columns:
    train_df['processed_text'] = train_df['text'].apply(lambda x: preprocess_text(x, method='lemmatization'))
else:
    print("Column 'text' not found in training data.")

if 'text' in test_df.columns:
    test_df['processed_text'] = test_df['text'].apply(lambda x: preprocess_text(x, method='lemmatization'))
else:
    print("Column 'text' not found in test data.")

processed_train_text = train_df['processed_text'].tolist() if 'processed_text' in train_df.columns else []
processed_test_text = test_df['processed_text'].tolist() if 'processed_text' in test_df.columns else []

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(texts, batch_size=100):
    all_embeddings = []
    num_batches = len(texts) // batch_size + (1 if len(texts) % batch_size > 0 else 0)

    print(f"Total number of batches: {num_batches}")

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        # Tokenize and encode the batch
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True)

        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Get mean of the last hidden state to represent the embeddings
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
        all_embeddings.extend(batch_embeddings)

        # Print status update
        print(f"Processed batch {i // batch_size + 1}/{num_batches}")

    return np.array(all_embeddings)

# Example usage
if processed_train_text:
    print("Generating embeddings for training data...")
    train_embeddings = get_bert_embeddings(processed_train_text)
    np.save('/content/train_embeddings.npy', train_embeddings)
    print("Training embeddings saved.")
else:
    print("No processed training text available.")

if processed_test_text:
    print("Generating embeddings for test data...")
    test_embeddings = get_bert_embeddings(processed_test_text)
    np.save('/content/test_embeddings.npy', test_embeddings)
    print("Test embeddings saved.")
else:
    print("No processed test text available.")


                                                text  label
0  Wall St. Bears Claw Back Into the Black (Reute...      2
1  Carlyle Looks Toward Commercial Aerospace (Reu...      2
2  Oil and Economy Cloud Stocks' Outlook (Reuters...      2
3  Iraq Halts Oil Exports from Main Southern Pipe...      2
4  Oil prices soar to all-time record, posing new...      2
                                                text  label
0  Fears for T N pension after talks Unions repre...      2
1  The Race is On: Second Private Team Sets Launc...      3
2  Ky. Company Wins Grant to Study Peptides (AP) ...      3
3  Prediction Unit Helps Forecast Wildfires (AP) ...      3
4  Calif. Aims to Limit Farm-Related Smog (AP) AP...      3


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Generating embeddings for training data...
Total number of batches: 1200
Processed batch 1/1200
Processed batch 2/1200
Processed batch 3/1200
Processed batch 4/1200
Processed batch 5/1200
Processed batch 6/1200
Processed batch 7/1200
Processed batch 8/1200
Processed batch 9/1200
Processed batch 10/1200
Processed batch 11/1200
Processed batch 12/1200
Processed batch 13/1200
Processed batch 14/1200
Processed batch 15/1200
Processed batch 16/1200
Processed batch 17/1200
Processed batch 18/1200
Processed batch 19/1200
Processed batch 20/1200
Processed batch 21/1200
Processed batch 22/1200
Processed batch 23/1200
Processed batch 24/1200
Processed batch 25/1200
Processed batch 26/1200
Processed batch 27/1200
Processed batch 28/1200
Processed batch 29/1200
Processed batch 30/1200
Processed batch 31/1200
Processed batch 32/1200
Processed batch 33/1200
Processed batch 34/1200
Processed batch 35/1200
Processed batch 36/1200
Processed batch 37/1200
Processed batch 38/1200
Processed batch 39/1200


In [ ]:
train_embeddings.shape,test_embeddings.shape

((120000, 768), (7600, 768))

In [ ]:
train_embeddings.shape

(120000, 768)